In [75]:
# 输入西瓜数据集
watermelon=[[0.697,0.460,'是'],[0.774,0.376,'是'],[0.634,0.264,'是'],
           [0.608,0.318,'是'],[0.556,0.215,'是'],[0.403,0.237,'是'],
           [0.481,0.149,'是'],[0.437,0.211,'是'],[0.666,0.091,'否'],
           [0.243,0.267,'否'],[0.245,0.057,'否'],[0.343,0.099,'否'],
           [0.639,0.161,'否'],[0.657,0.198,'否'],[0.360,0.370,'否'],
           [0.593,0.042,'否'],[0.719,0.103,'否']]

In [95]:
import numpy as np 

## 提取特征和类别
features_raw = watermelon[:,0:2].astype('float')
labels_raw = watermelon[:,-1]

# 数据预处理
x0 = np.ones((len(watermelon),1))
features = np.column_stack((x0,features_raw))

labels=list()
for i in range(len(labels_raw)):
    if labels_raw[i]=='是':
        labels.append(1)
    else:
        labels.append(0)
        
## 拆分训练集和验证集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size = 0.4, random_state=4)

In [125]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def gradient_Ascent(training_feature,training_label,alpha,max_iterations):
    m, n = training_feature.shape
    label = np.mat(training_label).reshape(m,1)
    
    weights = np.ones((n,1))
    for i in range(max_iterations):
        error = label - sigmoid(np.dot(training_feature,weights))
        weights = weights + alpha*np.dot(training_feature.T,error)
    return weights

def test_pred(test_feature, weights):
    pred = np.dot(test_feature, weights)
    pred_label = list()
    for i in range(len(pred)):
        if pred[i]>= 0.5:
            pred_label.append(1)
        else:
            pred_label.append(0)
    return pred_label

def test_score(pred_label,test_label):
    count = 0
    for i in range(len(pred_label)):
        if pred_label[i] == test_label[i]:
            count += 1
    accuracy_score = count/len(pred_label)
    return accuracy_score

In [147]:
weights=gradient_Ascent(X_train,y_train,alpha=5,max_iterations=3500)
pred_label=test_pred(X_test,weights)
accuracy_score=test_score(pred_label, y_test)
print(accuracy_score)

0.7142857142857143
